# Robot Maze example

In [2]:
import numpy as np
import pandas as pd
pd.set_option('precision', 3)

## Find the $V^\pi$

In [3]:
A = np.matrix(np.zeros((7,7)))
for i in range(7):
    if i != 2:
        A[i,i]=0.91 
    else:
        A[i,i]=1

for i in range(7):
    if i==0 or i==1:
        A[i,i+1]=-0.81
    elif i==3:
        A[i,0]=-0.81
    elif i != 2:
        A[i,i-1]=-0.81


B = np.matrix(np.zeros((7,1)))
B[2] = 1000
print(A)
print(B)

[[ 0.91 -0.81  0.    0.    0.    0.    0.  ]
 [ 0.    0.91 -0.81  0.    0.    0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.    0.  ]
 [-0.81  0.    0.    0.91  0.    0.    0.  ]
 [ 0.    0.    0.   -0.81  0.91  0.    0.  ]
 [ 0.    0.    0.    0.   -0.81  0.91  0.  ]
 [ 0.    0.    0.    0.    0.   -0.81  0.91]]
[[   0.]
 [   0.]
 [1000.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]]


In [4]:
V = np.linalg.inv(A)@B
print(V)

[[ 792.29561647]
 [ 890.10989011]
 [1000.        ]
 [ 705.23016411]
 [ 627.73234388]
 [ 558.75076763]
 [ 497.34958437]]


## Value Iteration

In [5]:
# States 1 - 7
S = list(range(1,8))
# Actions
A = { 'L', 'R', 'U', 'D' }
gamma = 0.9
# Reward
R = { 1: 0, 2: 0, 3: 1000, 4: 0, 5: 0, 6: 0, 7: 0}
# Psa
Psa = {
    1: {
        'L': { 1: 1., 2: .0, 3: .0, 4: .0, 5: .0, 6: .0, 7: .0},
        'R': { 1: .1, 2: .9, 3: .0, 4: .0, 5: .0, 6: .0, 7: .0},
        'U': { 1: 1., 2: .0, 3: .0, 4: .0, 5: .0, 6: .0, 7: .0},
        'D': { 1: .1, 2: .0, 3: .0, 4: .9, 5: .0, 6: .0, 7: .0}
    },
    2: {
        'L': { 1: .9, 2: .1, 3: .0, 4: .0, 5: .0, 6: .0, 7: .0},
        'R': { 1: .0, 2: .1, 3: .9, 4: .0, 5: .0, 6: .0, 7: .0},
        'U': { 1: .0, 2: 1., 3: .0, 4: .0, 5: .0, 6: .0, 7: .0},
        'D': { 1: .0, 2: 1., 3: .0, 4: .0, 5: .0, 6: .0, 7: .0}
    },
    3: { },
    4: {
        'L': { 1: .0, 2: .0, 3: .0, 4: 1., 5: .0, 6: .0, 7: .0},
        'R': { 1: .0, 2: .0, 3: .0, 4: 1., 5: .0, 6: .0, 7: .0},
        'U': { 1: .9, 2: .0, 3: .0, 4: .1, 5: .0, 6: .0, 7: .0},
        'D': { 1: .0, 2: .0, 3: .0, 4: .1, 5: .9, 6: .0, 7: .0}
    },
    5: {
        'L': { 1: .0, 2: .0, 3: .0, 4: .0, 5: 1., 6: .0, 7: .0},
        'R': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .1, 6: .9, 7: .0},
        'U': { 1: .0, 2: .0, 3: .0, 4: .9, 5: .1, 6: .0, 7: .0},
        'D': { 1: .0, 2: .0, 3: .0, 4: .0, 5: 1., 6: .0, 7: .0}
    },
    6: {
        'L': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .9, 6: .1, 7: .0},
        'R': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: .1, 7: .9},
        'U': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: 1., 7: .0},
        'D': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: 1., 7: .0}
    },
    7: {
        'L': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: .9, 7: .1},
        'R': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: .0, 7: 1.},
        'U': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: .0, 7: 1.},
        'D': { 1: .0, 2: .0, 3: .0, 4: .0, 5: .0, 6: .0, 7: 1.}
    },
}
terminal = { 1: False, 2: False, 3: True, 4: False, 5: False, 6: False, 7: False}

In [33]:
V = {s: 0 for s in S}
print(V)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0}


In [25]:
# V(s) <- 0
V = {s: 0 for s in S}
# Define "change" to check convergence
change = np.inf
iter = 0
# Policy
pi = {}

while change >1e-6:
    change = 0
    print("*************************************************************************")
    for s in S:
        print("============= state", s, "=============")
        max_val = 0
        max_a = None
        if not terminal[s]:
            for a in A:
                val = 0
                for sp in S:
                    # val = sum(s'E S)[Psa(s')V(s')]
                    val += Psa[s][a][sp] * V[sp]
                print(a, Psa[s][a], V, val)
                if max_a is None or val > max_val:
                    max_val = val
                    # Policy iteration (set pi[s] if val>max "eqaul to" argmax)
                    max_a = a
                    pi[s] = a
        else:
            print("Terminate")
            pi[s] = 'TERM'
        old_val = V[s]
        print("R(s):", R[s], " max val:", max_val)
        # V(s) <- R(s)+max_(aEA)[gamma*sum(s'E S)[Psa(s')V(s')]]
        V[s] = R[s] + gamma*max_val
        change += np.abs(V[s]-old_val)
        print("Vs", V[s], "old val", old_val, "change", change)
    #print('Iteration', iter, '| V = ', V)
    iter += 1

print('Optimal policy:', pi)


*************************************************************************
============= state 1 =============
L {1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
D {1: 0.1, 2: 0.0, 3: 0.0, 4: 0.9, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
R {1: 0.1, 2: 0.9, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
U {1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
R(s): 0  max val: 0.0
Vs 0.0 old val 0 change 0.0
============= state 2 =============
L {1: 0.9, 2: 0.1, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0.0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
D {1: 0.0, 2: 1.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0.0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
R {1: 0.0, 2: 0.1, 3: 0.9, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0} {1: 0.0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0} 0.0
U {1: 0.0, 2: 1.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.